In [1]:
## utils

import exiftool
import numpy as np

def get_metadata(im_path):
    with exiftool.ExifToolHelper() as et:
       metadata = et.get_metadata(im_path)
       return metadata[0]
        #print(metadata[0].get("EXIF:GPSLatitude"))

def position(self): 
        """get the WGS-84 latitude, longitude tuple as signed decimal degrees"""
        lat = self.get_item('EXIF:GPSLatitude')
        latref = self.get_item('EXIF:GPSLatitudeRef')
        if latref == 'S':
            lat *= -1.0
        lon = self.get_item('EXIF:GPSLongitude')
        lonref = self.get_item('EXIF:GPSLongitudeRef')
        if lonref == 'W':
            lon *= -1.0
        alt = self.get_item('EXIF:GPSAltitude')
        return lat, lon, alt

def get_gps_coordinates(metadata):

    latitude = metadata.get('EXIF:GPSLatitude')
    latref = metadata.get('EXIF:GPSLatitudeRef')
    if latref == 'S':
        latitude *= -1.0
    longitude = metadata.get('EXIF:GPSLongitude')
    lonref = metadata.get('EXIF:GPSLongitudeRef')
    if lonref == 'W':
        longitude *= -1.0

    #latitude = metadata.get("EXIF:GPSLatitude")
    #longitude = metadata.get("EXIF:GPSLongitude")
    return latitude, longitude

def get_image_resolution(metadata):
    image_width = metadata.get("EXIF:ExifImageWidth") or metadata.get( "EXIF:ImageWidth")
    image_heigth = metadata.get("EXIF:ExifImageHeight") or metadata.get( "EXIF:ImageHeight")

    return image_width, image_heigth

def calculate_gsd(metadata):
    relative_altitude_text = metadata["XMP:RelativeAltitude"]
    print("relative_altitude_text:", relative_altitude_text)
    signo, numero = (relative_altitude_text[0], relative_altitude_text[1:]) if relative_altitude_text[0] in '+-' else ("+", relative_altitude_text[0])
    altitude_drone = float(numero) if signo == '+' else -float(numero)
    altitude_drone = altitude_drone * 100 # cm/pixel
    #print("altitude_drone:", altitude_drone)
    image_width, image_heigth = get_image_resolution(metadata)

    fov_metadata = metadata.get('Composite:FOV')
    if  isinstance(fov_metadata, str):
    # Extraer el FOV angular (primer número)
        fov_metadata = fov_metadata.split()[0]
        fov_metadata = float(fov_metadata)
    
    fov = float(fov_metadata)

    #print("fov:", fov)
    fov_rad = np.radians(fov)
    rel_asp = image_width / image_heigth
    fov_h = 2 * np.arctan((rel_asp / (np.sqrt(rel_asp ** 2 + 1))) * np.tan(fov_rad / 2))
    fov_v = 2 * np.arctan((1 / (np.sqrt(rel_asp ** 2 + 1))) * np.tan(fov_rad / 2))

    GSD_horizontal = (2 * altitude_drone * np.tan(fov_h)/ 2) / image_width #  cm/pixel
    GSD_vertical = (2 * altitude_drone * np.tan(fov_v / 2)) / image_heigth 
    
    return GSD_horizontal, GSD_vertical

def get_drone_model(metadata):
    if metadata["EXIF:Make"] == "DJI":
        return metadata["EXIF:Model"]
    return None

folder_path = "./data/trees-avocado/m3m/campo2/images/"


im_path = folder_path + "DJI_20241128154239_0001_D.JPG"
metadata = get_metadata(im_path)
latitude, longitude = get_gps_coordinates(metadata)
image_width, image_heigth = get_image_resolution(metadata)
yaw_degree = metadata.get("XMP:GimbalYawDegree")
pitch_degree = metadata.get("XMP:GimbalPitchDegree")
roll_degree = metadata.get("XMP:GimbalRollDegree")

if not isinstance(yaw_degree, float):
    signo, yaw_degree = (yaw_degree[0], yaw_degree[1:]) if yaw_degree[0] in '+-' else ("+", yaw_degree[0])
    yaw_degree =  float(yaw_degree) if signo == '+' else -float(yaw_degree)

GSD_horizontal, GSD_vertical = calculate_gsd(metadata)

model_drone = get_drone_model(metadata)
#GSD = 0.78
print(f"Latitud: {latitude}, Longitud: {longitude}")
print(f"image_width: {image_width}, image_heigth: {image_heigth}")
print(f"GSD_horizontal: {GSD_horizontal}, GSD_vertical: {GSD_vertical}")
print(f"yaw_degree: {yaw_degree}")
print(f"model_drone: {model_drone}")

ExifToolExecuteError: execute returned a non-zero exit status: 1

In [ ]:
camera_features = {
    "M3M": {
        "bands_features": {
                "RGB": { 
                    "suffix": "_D",
                    "file": "jpg"
                },
                "GREEN": { 
                "suffix": "_MS_G",
                "file": "tif"
                },
                'RED': { 
                "suffix": "_MS_R",
                "file": "tif"
                } ,
                "NIR": { 
                "suffix": "_MS_NIR",
                "file": "tif"
                },
                'RED_EDGE': { 
                "suffix": "_MS_RE",
                "file": "tif"
                }}
    }
}

bands_features = camera_features[model_drone]["bands_features"]
bands_features

In [ ]:
from glob import glob
import os
import pandas as pd
#dir_images = "./mavic3m/campo2/Viaje3-27-Nov/DJI_202411281538_002_acco2-campo2/DJI_202411281538_002_acco2-campo2"
#folder_path = "mavic3m/campo2/Viaje3-27-Nov/DJI_202411281538_002_acco2-campo2/"

tif_files = glob(f"{folder_path}/*.TIF")

jpg_files = glob(f"{folder_path}/*.JPG")


images_names = tif_files + jpg_files
print(f"{len(jpg_files)} imagenes encontradas")

im_basenames = ["_".join(os.path.basename(im_file).split("_")[:3]) for im_file in jpg_files]

latitudes = []
longitudes  = []
yaw_degrees  = []
pitch_degrees  = []
roll_degrees = []
images_w = []
images_h = []
images_GSDH = []
images_GSDV = [] 
datetimes = []

for im_name in im_basenames:
    ## solo usamos el RGB por ahora
    im_path = f"{folder_path}/{im_name}{bands_features['RGB']['suffix']}.JPG"
    metadata = get_metadata(im_path)
    latitude, longitude = get_gps_coordinates(metadata)
    print(f"Latitud: {latitude}, Longitud: {longitude}")
    image_width, image_height = get_image_resolution(metadata)
    yaw_degree = metadata.get("XMP:GimbalYawDegree")
    pitch_degree = metadata.get("XMP:GimbalPitchDegree")
    roll_degree = metadata.get("XMP:GimbalRollDegree")
    if not isinstance(yaw_degree, float):
        signo, yaw_degree = (yaw_degree[0], yaw_degree[1:]) if yaw_degree[0] in '+-' else ("+", yaw_degree[0])
        yaw_degree =  float(yaw_degree) if signo == '+' else -float(yaw_degree)

    GSD_horizontal, GSD_vertical = calculate_gsd(metadata)
    
    latitudes.append(latitude)
    longitudes.append(longitude)
    yaw_degrees.append(yaw_degree)
    pitch_degrees.append(pitch_degree)
    roll_degrees.append(roll_degree)
    images_w.append(image_width)
    images_h.append(image_height)
    images_GSDH.append(GSD_horizontal)
    images_GSDV.append(GSD_vertical)
    datetimes.append(metadata.get("EXIF:DateTimeOriginal"))

df_images_metadata =  pd.DataFrame({"basename": im_basenames,
                                    "latitude": latitudes,
                                    "longitude" : longitudes,
                                    "yaw_degree": yaw_degrees,
                                    "pitch_degree": pitch_degrees,
                                    "roll_degree": roll_degrees,
                                    "image_w": images_w,
                                    "image_h": images_h,
                                    "GSDH": images_GSDH,
                                    'GSDV': images_GSDV,
                                    "DateTimeOriginal": datetimes,
                                    })
df_images_metadata["Timestamp_ms"] = pd.to_datetime(df_images_metadata["DateTimeOriginal"], format="%Y:%m:%d %H:%M:%S").astype(int) // 10**6

def filter_oblique_images(df_images):
    return df_images[(df_images['pitch_degree'] < -89) & (df_images['pitch_degree'] > -91)]

df_images_filtered = filter_oblique_images(df_images_metadata)

df_images_filtered.to_csv("./df_images_metadata.csv", index= False)
df_images_filtered = df_images_filtered.sort_values(by="Timestamp_ms", ascending=True)
df_images_filtered.head(10)

In [ ]:
## Mostrar mapa imagenes

import plotly.express as px
import plotly.graph_objects as go
#longitudes, latitudes = zip(*images_coordinates)
mapbox_token = "pk.eyJ1IjoiYW50aG9ueW1nMSIsImEiOiJjbTNuajBzamwxZXMxMmtweDV3anZkcHRxIn0.1ZlgQwJcn4msckpzTNSSJg"


fig = px.scatter_mapbox(
    lat=df_images_filtered["latitude"],  # Latitud de ejemplo (San Francisco)
    lon=df_images_filtered["longitude"],  # Longitud de ejemplo (San Francisco)
    zoom=18,  # Nivel de zoom
    height=800,  # Altura del gráfico
    hover_name= df_images_filtered["basename"]
)

# Configurar el mapa con el estilo satelital
fig.update_layout(
    mapbox_style="satellite",  # Estilo de mapa satelital
    mapbox_accesstoken=mapbox_token  # Tu token de acceso Mapbox
)


# Mostrar el mapa
fig.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detection_trees_pdi(image_uav, show = False):
    image_uav = cv2.GaussianBlur(image_uav, (5,5), 0)
    im_lab = cv2.cvtColor(image_uav, cv2.COLOR_BGR2HSV)

    
    rango_min = np.array([35, 90, 0])
    rango_max = np.array([70, 255, 255])

    #rango_min = np.array([40, 60, 20])
    #rango_max = np.array([80, 255, 200])

    kernel = np.ones((5,5), np.uint8)

    mascara = cv2.inRange(im_lab, rango_min, rango_max)
    mascara = cv2.morphologyEx(mascara, cv2.MORPH_OPEN, kernel, iterations = 1)
    mascara = cv2.morphologyEx(mascara, cv2.MORPH_CLOSE, kernel, iterations= 3)
    #mascara = cv2.dilate(mascara, kernel , iterations = 1)
    #mascara = cv2.erode(mascara, kernel , iterations = 2)
    #mascara = cv2.dilate(mascara, kernel , iterations = 1)
    countorus, _  = cv2.findContours(mascara, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    final_contours = []
    bboxes = []
    masks = []
    im_bboxes = image_uav.copy()

    for cn in countorus:
        area_cn = cv2.contourArea(cn)
        if area_cn > 65000:
            final_contours.append(cn)
            x, y, w, h = cv2.boundingRect(cn)
            image_mask_tree = np.zeros_like(mascara, dtype= mascara.dtype)
            cv2.drawContours(image_mask_tree, [cn], -1, color = 255, thickness = cv2.FILLED)
            masks.append(image_mask_tree)
            bboxes.append([x,y,w,h])
            #print("area:", area_cn)
            cv2.rectangle(im_bboxes, (x,y), (x + w, y + h), (255,0,0), 14)

    if show:
        mascara_filtrada = np.zeros_like(mascara, np.uint8)
        cv2.drawContours(mascara_filtrada, final_contours, -1, color = 255, thickness = cv2.FILLED)
        fig, axis = plt.subplots(2,2 , figsize = (15,8))
        axis[0][0].imshow(image_uav)
        axis[0][1].imshow(mascara, cmap = 'gray')
        axis[1][0].imshow(im_bboxes)
        axis[1][1].imshow(mascara_filtrada, cmap = 'gray')

    return bboxes, masks

im_trees = cv2.imread(im_path)

bboxes, masks = detection_trees_pdi(im_trees, show = True)

In [ ]:
import os

def detection_trees_human(im_path, show = False):
    image_uav = cv2.imread(im_path)
    mask_name = os.path.basename(im_path)[:-4]
    print("mask_name: ", mask_name)
    mask_path = f"./data/trees-avocado/m3m/campo2/masks/{mask_name}_MASK.JPG"
    print("mask_path:", mask_path)

    image_uav = cv2.GaussianBlur(image_uav, (5,5), 0)
    mask = cv2.imread(mask_path)
    print("mask:", mask.shape)
    mask = mask != 0
    mask = mask.astype(np.uint8)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations = 1)
    #mask = cv2.erode(mask, kernel , iterations = 3)
    #mask = cv2.Canny(mask, 0, 500)
    countorus, _  = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    final_contours = []
    bboxes = []
    masks = []
    im_bboxes = image_uav.copy()

    for cn in countorus:
        area_cn = cv2.contourArea(cn)
        if area_cn > 65000:
            final_contours.append(cn)
            x, y, w, h = cv2.boundingRect(cn)
            bboxes.append([x,y,w,h])
            image_mask_tree = np.zeros_like(mask, dtype= mask.dtype)
            cv2.drawContours(image_mask_tree, [cn], -1, color = 255, thickness = cv2.FILLED)
            masks.append(image_mask_tree)
            #print("area:", area_cn)
            cv2.rectangle(im_bboxes, (x,y), (x + w, y + h), (255,0,0), 14)
    print("Numero de contornos:", len(final_contours))
    if show:
        mascara_filtrada = np.zeros_like(mask, np.uint8)
        cv2.drawContours(mascara_filtrada, final_contours, -1, color = 255, thickness = cv2.FILLED)
        fig, axis = plt.subplots(2,2 , figsize = (15,8))
        axis[0][0].imshow(image_uav)
        axis[0][1].imshow(mask, cmap = 'gray')
        axis[1][0].imshow(im_bboxes)
        axis[1][1].imshow(mascara_filtrada, cmap = 'gray')

    return bboxes, masks

#im_trees = cv2.imread(im_path4)
#mask_path = "data/trees-avocado/masks/DJI_20241017094717_0085_MASK.JPG"
#mask_path = "data/trees-avocado/masks/DJI_20241017094703_0071_MASK.JPG"


bboxes, masks = detection_trees_human(im_path , show = True)

In [ ]:
def crop_trees(image, bboxes, masks):
    # crop tress
    trees_crops = []
    seg_trees = []
    masks_crops = []
    centers = []
    for bbox, mask in zip(bboxes, masks):
        x, y, w, h = bbox
        n_w = int(w * 1.2)
        n_h = int(h * 1.2)
        x = max(x - (n_w - w) // 2 , 0)
        y = max(y - (n_h - h) // 2 , 0)
        im_frame = image[y:y + n_h, x: x + n_w, :]
        seg_tree = np.zeros_like(im_frame, dtype = im_frame.dtype)
        seg_tree = cv2.bitwise_and(image, image, mask = mask)
        trees_crops.append(im_frame)
        masks_crops.append(mask[y: y + n_h, x: x + n_w])
        seg_trees.append(seg_tree[y: y + n_h, x: x + n_w, :])

        centers.append([(2 * x + n_w) // 2,  (2 * y + n_h) // 2,])

    return trees_crops, seg_trees, masks_crops, centers

trees_crops, seg_trees, masks_trees, centers = crop_trees(im_trees, bboxes, masks)

idx = 5
fig, axis = plt.subplots(1,3, figsize = (15, 9))
im_tree = trees_crops[idx]
im_seg_tree = seg_trees[idx]
mask_tree = masks_trees[idx]
axis[0].imshow(im_tree)
axis[1].imshow(im_seg_tree)
axis[2].imshow(mask_tree, cmap = "gray")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, segmentation, color
from skimage.util import img_as_float
from skimage.segmentation import slic, mark_boundaries
from PIL import Image

# Aplicar SLIC
n_segments = 5000  # Número de superpíxeles
compactness = 20   # Parámetro de compactación
segments = slic(im_seg_tree, n_segments=n_segments, compactness=compactness, start_label=1)

slic_boundaries = mark_boundaries(im_seg_tree, segments)
boundaries = np.array((slic_boundaries * 255.0).astype(np.uint8))

im_boundaries = Image.fromarray(boundaries)
im_boundaries.save("leaf_slic_boundaries.jpg")

# Mostrar la imagen segmentada
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Imagen original
ax[0].imshow(im_seg_tree)
ax[0].set_title("Imagen Original")
ax[0].axis("off")

# Imagen con bordes de superpíxeles
ax[1].imshow(boundaries)
ax[1].set_title("Superpíxeles con SLIC")
ax[1].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
im_center_rot = im_trees.copy()

for i, c in enumerate(centers):
    cv2.circle(im_center_rot, c, radius = 50, color = (255, 0, 0), thickness = -1)
    cv2.putText(im_center_rot, str(i + 1), (c [0] - 200, c[1] + 10), cv2.FONT_HERSHEY_SIMPLEX, 12, (0,0,255), 15 )

plt.imshow(im_center_rot)

In [ ]:
import matplotlib.pyplot as plt
## IMAGE ROTADA SEGUN PERSPECTIVA DEL DRONE CON EL NORTE
from PIL import Image

img = Image.fromarray(im_center_rot)
roteted_img = img.rotate(-yaw_degree, expand=True)
#roteted_img

fig, axis = plt.subplots(1, 2, figsize=(20,12))

latitudes, longitudes = zip(*coordinates[:])

axis[0].imshow(roteted_img)
axis[1].scatter(longitudes , latitudes)
for i, (x, y) in enumerate(zip(longitudes, latitudes)):
    axis[1].text(x, y, str(i + 1), fontsize = 12, ha = "right", color = "darkred")
axis[1].scatter(longitude, latitude, color = "red")
axis[1].set_xlabel("longitud")
axis[1].set_ylabel("latitud")
axis[1].ticklabel_format(useOffset=False, style='plain', axis='both')
axis[1].grid(True)

In [ ]:
from tqdm import tqdm
from glob import glob
import tempfile
import json
#images_path = glob("data/trees-avocado/images/*.JPG")

images_path = glob("data/campo1/vuelo2/RGB/*.JPG")

def save_img(image, path_save):
    img = Image.fromarray(image)
    img.save(path_save)

def save_json(data, path_save):
    with open(path_save, "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

def extraction_trees(im_path):
    # Extracion de matadatos
    metadata = get_metadata(im_path)
    latitude, longitude = get_gps_coordinates(metadata)
    image_width, image_heigth = get_image_resolution(metadata)
    yaw_degree = metadata.get("XMP:GimbalYawDegree")
    
    if not isinstance(yaw_degree, float):
        signo, yaw_degree = (yaw_degree[0], yaw_degree[1:]) if yaw_degree[0] in '+-' else ("+", yaw_degree[0])
        yaw_degree =  float(yaw_degree) if signo == '+' else -float(yaw_degree)
    GSD_horizontal, GSD_vertical = calculate_gsd(metadata)
    
    # Segmentacion de Arboles
    #bboxes, masks = detection_trees_human(im_path , show = False)
    image_uav = cv2.imread(im_path)
    bboxes, masks = detection_trees_pdi(image_uav, show = False)
    # Recorte de los arboles
    imgs_trees, segs_trees, masks_trees, centers = crop_trees(image_uav, bboxes, masks)

    coordinates = [calculate_gps_for_pixel(c , 
                                           (latitude, longitude), 
                                           GSD_horizontal, GSD_vertical, 
                                           (image_heigth, image_width), 
                                           yaw_degree) 
               for c in centers[:]]
     #im_center_rot = im_trees.copy()

    return imgs_trees, segs_trees, masks_trees, coordinates, centers

#trees = []
#segs = []
#masks = []
coordinates = []
centers_ref_images = []

id_detect = 0
#with tempfile.TemporaryDirectory() as temp_dir:
temp_dir = "./temp_dir"
os.makedirs(temp_dir, exist_ok = True)
print(f"Parte guardada en: {temp_dir}")

for im_path in tqdm(images_path[], desc = "Images"):
    imgs_trees, segs_trees, masks_trees, coordinates_trees, centers_ref = extraction_trees(im_path)
    #print(f"{len(imgs_trees)} arboles detectados")
    coordinates.extend(coordinates_trees)
    centers_ref_images.append(centers_ref)
    for tree, seg, mask in zip(imgs_trees, segs_trees, masks_trees):
        save_img(tree, f"{temp_dir}/Tree-D{id_detect}.png")
        save_img(seg, f"{temp_dir}/Seg-D{id_detect}.png")
        save_img(mask, f"{temp_dir}/Mask-D{id_detect}.png")
        id_detect+=1

# Leer el archivo JSON

with open(f"{temp_dir}/coordinates.json", "w") as file:
    json.dump(coordinates, file)

# Guardar en un archivo JSON
with open(f"{temp_dir}/centers_ref_images.json", "w") as file:
    json.dump(centers_ref_images, file)

In [ ]:
import json
temp_dir = "./v1_campo1_temp_dir"

with open(f"{temp_dir}/coordinates.json", "r") as file:
    coordinates = json.load(file)

with open(f"{temp_dir}/centers_ref_images.json", "r") as file:
    centers_ref_images = json.load(file)

In [ ]:
import matplotlib.pyplot as plt

points_neighbords = [[-13.787972834900415, -72.95041594796696], [-13.78796284393943, -72.95042109626344], [-13.787965130887258, -72.9504088790114], [-13.787966337393788, -72.9504085358206], [-13.787960483740502, -72.9504052058341], [-13.787965203959878, -72.95040334693887]]


colores = [
    "darkblue", "orange", "teal", "magenta", "gold", 
    "indigo", "lime", "turquoise", "orchid", "coral", 
    "crimson", "cyan", "darkgreen", "deeppink", "lavender", 
    "navy", "olive", "peru", "plum", "salmon", 
    "sienna", "silver", "slateblue", "springgreen", "steelblue", 
    "tan", "thistle", "tomato", "violet", "wheat", 
    "aquamarine", "azure", "beige", "bisque", "blueviolet", 
    "brown", "chartreuse", "chocolate", "cornflowerblue", "darkcyan", 
    "darkgoldenrod", "darkkhaki", "darkmagenta", "darkorange", "darkorchid", 
    "darksalmon", "darkseagreen", "darkslateblue", "darkturquoise", "firebrick",
    "forestgreen", "gainsboro", "ghostwhite", "greenyellow", "honeydew", 
    "hotpink", "ivory", "khaki", "lavenderblush", "lawngreen", 
    "lemonchiffon", "lightblue", "lightcoral", "lightcyan", "lightgoldenrodyellow", 
    "lightgreen", "lightgrey", "lightpink", "lightsalmon", "lightseagreen", 
    "lightskyblue", "lightslategray", "lightsteelblue", "lightyellow", "limegreen", 
    "linen", "mediumaquamarine", "mediumblue", "mediumorchid", "mediumpurple", 
    "mediumseagreen", "mediumslateblue", "mediumspringgreen", "mediumturquoise", "mediumvioletred", 
    "midnightblue", "mintcream", "mistyrose", "moccasin", "navajowhite", 
    "oldlace", "olivedrab", "orangered", "palegoldenrod", "palegreen", 
    "paleturquoise", "palevioletred", "papayawhip", "peachpuff", "powderblue"
]

point = [-13.787966897188113, -72.95041217231403]
point_neighbord = [-13.787972834900415, -72.95041594796696]

points_neighbords_neighbord = [[-13.787966897188113, -72.95041217231403], [-13.78796284393943, -72.95042109626344], [-13.787965130887258, -72.9504088790114], [-13.787966337393788, -72.9504085358206], [-13.78797857963158, -72.95042796878762], [-13.787974538098219, -72.95042396718758], [-13.787984029495078, -72.9504161528893]]

fig = plt.figure(figsize=(22, 15))

plt.scatter(point[1] , point[0])
plt.text(point[1], point[0], str('P'), fontsize = 15, ha = "right", color = colores[0])

latitudes, longitudes = zip(*points_neighbords[:])
plt.scatter(longitudes , latitudes)
for j, (x, y) in enumerate(zip(longitudes, latitudes)):
    plt.text(x, y, str(j + 1), fontsize = 15, ha = "right", color = colores[1])

plt.ticklabel_format(useOffset=False, style='plain', axis='both')

plt.scatter(point_neighbord[1] , point_neighbord[0])
plt.text(point_neighbord[1] - 0.0000001, point_neighbord[0], str('N'), fontsize = 15, ha = "right", color = colores[2])

latitudes, longitudes = zip(*points_neighbords_neighbord[:])
plt.scatter(longitudes , latitudes)
for j, (x, y) in enumerate(zip(longitudes, latitudes)):
    plt.text(x - 0.0000004 , y, 'NN' + str(j + 1), fontsize = 15, ha = "right", color = colores[3])

plt.ticklabel_format(useOffset=False, style='plain', axis='both')


plt.grid(True)
# Mostrar la figura completa
plt.tight_layout()
plt.show()

In [ ]:
colores = [
    "darkblue", "orange", "teal", "magenta", "gold", 
    "indigo", "lime", "turquoise", "orchid", "coral", 
    "crimson", "cyan", "darkgreen", "deeppink", "lavender", 
    "navy", "olive", "peru", "plum", "salmon", 
    "sienna", "silver", "slateblue", "springgreen", "steelblue", 
    "tan", "thistle", "tomato", "violet", "wheat", 
    "aquamarine", "azure", "beige", "bisque", "blueviolet", 
    "brown", "chartreuse", "chocolate", "cornflowerblue", "darkcyan", 
    "darkgoldenrod", "darkkhaki", "darkmagenta", "darkorange", "darkorchid", 
    "darksalmon", "darkseagreen", "darkslateblue", "darkturquoise", "firebrick",
    "forestgreen", "gainsboro", "ghostwhite", "greenyellow", "honeydew", 
    "hotpink", "ivory", "khaki", "lavenderblush", "lawngreen", 
    "lemonchiffon", "lightblue", "lightcoral", "lightcyan", "lightgoldenrodyellow", 
    "lightgreen", "lightgrey", "lightpink", "lightsalmon", "lightseagreen", 
    "lightskyblue", "lightslategray", "lightsteelblue", "lightyellow", "limegreen", 
    "linen", "mediumaquamarine", "mediumblue", "mediumorchid", "mediumpurple", 
    "mediumseagreen", "mediumslateblue", "mediumspringgreen", "mediumturquoise", "mediumvioletred", 
    "midnightblue", "mintcream", "mistyrose", "moccasin", "navajowhite", 
    "oldlace", "olivedrab", "orangered", "palegoldenrod", "palegreen", 
    "paleturquoise", "palevioletred", "papayawhip", "peachpuff", "powderblue"
]


fig = plt.figure(figsize=(22, 15))

for i in range(100):
    latitudes, longitudes = zip(*groups[i])
    plt.scatter(longitudes , latitudes)
    for j, (x, y) in enumerate(zip(longitudes, latitudes)):
        plt.text(x, y, str(j + 1), fontsize = 15, ha = "right", color = colores[i])

    plt.ticklabel_format(useOffset=False, style='plain', axis='both')


plt.grid(True)
# Mostrar la figura completa
plt.tight_layout()
plt.show()

In [ ]:
import json

def select_point_by_distance_center(group_idx, centers, img_center_ref):
    if len(group_idx) == 1:
        return group_idx[0]
    
    distances = [euclidean_distance(cent, img_center_ref) for cent in centers]

    idx = np.argmin(distances)
    return group_idx[idx]

def save_img(image, path_save):
    img = Image.fromarray(image)
    img.save(path_save)

def save_json(data, path_save):
    with open(path_save, "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

centers_trees = [cent for centers_trees_img in centers_ref_images for cent in centers_trees_img]
img_center = image_width // 2, image_heigth // 2


filtered_trees_idx = [select_point_by_distance_center(group_idx, 
                                                      [centers_trees[idx] for idx in group_idx], 
                                                      img_center) 
                        for group_idx in groups_idx]


filtered_coordinates = [coordinates[idx] for idx in filtered_trees_idx]
names = [f"T{i+1}" for i in range(len(filtered_trees_idx))]

print(f"{len(filtered_trees_idx)} arboles encontrados")

In [ ]:
fig = plt.figure(figsize=(22, 15))


latitudes, longitudes = zip(*filtered_coordinates[0:])
plt.scatter(longitudes , latitudes)
for i, (x, y) in enumerate(zip(longitudes, latitudes)):
    plt.text(x, y, str(i + 1), fontsize = 15, ha = "right", color = "purple")
plt.ticklabel_format(useOffset=False, style='plain', axis='both')


plt.grid(True)
# Mostrar la figura completa
plt.tight_layout()
plt.show()